In [23]:
tokenizer.convert_tokens_to_ids(["GGGGGG"])

[4100]

In [27]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import pandas as pd

dataset = load_from_disk("../batch")

tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision="downstream_species_lm")

# 5447 is the highest token id
start_species_ids = tokenizer.convert_tokens_to_ids(["GGGGGG"])[0]

def add_species_input(dataset):
    # add species column
    df = pd.DataFrame(dataset)
    def gen_species_list(r):
        print(r)
        species = r["species_name"]
        species_id = tokenizer.convert_tokens_to_ids([species])[0]
        species_id -= start_species_ids
        num_rows = len(r["input_ids"])
        species_id = [species_id] * num_rows
        return species_id


    df["species_id"] = df.apply(lambda r : gen_species_list(r), axis=1)
    return Dataset.from_pandas(df)

dataset["train"] = add_species_input(dataset["train"])
dataset["test"] = add_species_input(dataset["test"])

dataset.save_to_disk("../batch_species")

species_name                         fonsecaea_monophora_gca_001642475
__index_level_0__                                              8623770
input_ids            [2, 4944, 1240, 849, 3381, 1221, 773, 3078, 12...
token_type_ids       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 0, dtype: object
species_name                  pseudogymnoascus_sp_24mn13_gca_001662595
__index_level_0__                                              4541058
input_ids            [2, 4534, 1846, 3276, 803, 3200, 499, 1984, 38...
token_type_ids       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
attention_mask       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: 1, dtype: object
species_name               fistulina_hepatica_atcc_64428_gca_000934395
__index_level_0__                                              5838157
input_ids            [2, 4669, 2936, 3537, 1846, 3273, 791, 3151, 3...
token_type_ids       [0, 0, 0, 

Saving the dataset (0/1 shards):   0%|          | 0/16 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16 [00:00<?, ? examples/s]

In [65]:
type(dataset)

datasets.arrow_dataset.Dataset